<a href="https://colab.research.google.com/github/JieShenAI/torch/blob/main/transformer/code/atttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
from torch import nn
import math

两个shape为(n,d)的矩阵相乘之后的结果矩阵，除以sqrt(n)之后的标准差才为1

sqrt(dk),dk就是两个矩阵相乘，消失的那个维度

In [ ]:
def get_attention_thita():
  d = 10 ** 3
  n = 10 ** 4
  a = torch.randn(n,d)
  print("a_var:",a.var())
  b = torch.randn(n,d)
  print("b var:",b.var())
  res = torch.matmul(a,b.t()) / (d ** 0.5)
  print("res.shape:",res.shape,"res.mean:",res.mean(),"res.std:",res.var())
get_attention_thita()

a_var: tensor(1.0006)
b var: tensor(1.0000)
res.shape: torch.Size([10000, 10000]) res.mean: tensor(-2.2428e-05) res.std: tensor(1.0007)


attention 通用函数

In [ ]:
def attention(query, key, value):
  d_k = query.size(-1)
  scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
  p_attn = scores.softmax(dim=-1)
  return torch.matmul(p_attn, value)

decoder的掩码

## 框架

transformer黑盒讲解: https://blog.csdn.net/zhaohongfei_358/article/details/126019181

### embedding的用法

参考链接：https://blog.csdn.net/zhaohongfei_358/article/details/122809709

nn.Embedding中的参数会参与梯度下降的。更新模型参数也会更新nn.Embedding的参数，或者说nn.Embedding的参数本身也是模型参数的一部分。

nn.Embeddding接受两个重要参数：

* num_embeddings：字典的大小。
* embedding_dim：要将单词编码成多少维的向量

例如，我们现在词典大小为20，现在要对hello, i, am，这三个单词进行编码，想将其编码为5维向量，则对应代码为：

一般0表示句子开始(\<bos>)，1表示句子结束(\<eos>)，2为填充(\<pad>)

In [6]:
embedding = nn.Embedding(20, 5, padding_idx=2) # 加上unknown对应的索引是2
# 3,4,5 代表 hello,i,am
token = embedding(torch.LongTensor([0,3,3,4,5,1,2,2]))
token,token.shape

(tensor([[-1.0356,  0.3127, -1.1603, -0.4334, -1.9822],
         [-0.1711,  0.1356,  0.6236, -1.1335, -1.0227],
         [-0.1711,  0.1356,  0.6236, -1.1335, -1.0227],
         [ 1.3297, -0.1141,  1.3465,  1.1570,  1.0736],
         [-0.9400,  0.4222, -1.2418,  1.7816,  2.1100],
         [-0.1921, -0.3887,  0.9072, -1.1672, -0.9567],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
        grad_fn=<EmbeddingBackward0>),
 torch.Size([8, 5]))

In [12]:
embedding.weight

Parameter containing:
tensor([[-1.0356,  0.3127, -1.1603, -0.4334, -1.9822],
        [-0.1921, -0.3887,  0.9072, -1.1672, -0.9567],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.1711,  0.1356,  0.6236, -1.1335, -1.0227],
        [ 1.3297, -0.1141,  1.3465,  1.1570,  1.0736],
        [-0.9400,  0.4222, -1.2418,  1.7816,  2.1100],
        [ 1.3845, -0.0335,  0.4945, -2.1824, -0.0474],
        [ 0.3450, -1.8370, -0.2142,  1.9349, -0.2536],
        [ 0.7656, -1.7489, -0.9148,  0.5867, -0.6040],
        [ 1.0515,  0.5338, -0.1564, -0.7515, -1.8969],
        [ 0.2011, -0.2918,  1.2682, -0.4076, -1.9063],
        [-0.4986,  0.3295, -0.1114, -0.0669, -0.1351],
        [-0.2832, -0.8711, -0.6945,  1.8596, -0.0973],
        [ 1.0646,  1.1381,  0.4273,  1.4794, -1.1820],
        [ 0.9517, -1.1829, -2.3106, -0.7703,  0.3045],
        [-0.2638,  0.7344, -1.9962, -1.0425, -1.7532],
        [-1.5127,  0.0668,  2.4475, -0.4815, -0.0954],
        [-0.6959,  0.7758,  1.2652,  0.1021

**相当于把输入的Tensor作为下标，从embedding的权重矩阵中取值**

指定`padding_idx`后，embedding在`padding_idx`的权重为0



---



### nn.Transformer

API文档: https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html

In [36]:
def transformer_demo():
  # 定义编码器，词典大小为10，要把token编码成128维的向量
  embedding = nn.Embedding(10, 128)
  # 定义transformer，模型维度为128（也就是词向量的维度）
  transformer = nn.Transformer(d_model=128, batch_first=True) # batch_first一定不要忘记
  # 定义源句子，可以想想成是 <bos> 我 爱 吃 肉 和 菜 <eos> <pad> <pad>
  src = torch.LongTensor([[0, 3, 4, 5, 6, 7, 8, 1, 2, 2]])
  
  # 定义目标句子，可以想想是 <bos> I like eat meat and vegetables <eos> <pad>
  tgt = torch.LongTensor([[0, 3, 4, 5, 6, 7, 8, 1, 2]])
  
  # jie: 我个人总觉得这个tgt应该是：
  # tgt = torch.LongTensor([[3, 4, 5, 6, 7, 8, 1, 2, 2]])

  # 将token编码后送给transformer（这里暂时不加Positional Encoding）
  outputs = transformer(embedding(src), embedding(tgt))
  f = lambda x: str(x.shape)
  print("embedding(src).size: %s\nembedding(tgt).size: %s\noutputs.size: %s" % (f(embedding(src)),f(embedding(tgt)),f(outputs)))
        
transformer_demo()


embedding(src).size: torch.Size([1, 10, 128])
embedding(tgt).size: torch.Size([1, 9, 128])
outputs.size: torch.Size([1, 9, 128])


`nn.Transformer`输出的shape与embedding(tgt)的shape保持一致。

这个 `nn.Transformer`，如何使用损失函数?

#### 掩码

掩码如何使用?

In [35]:
nn.Transformer.generate_square_subsequent_mask(3)

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])

验证掩码真的生效了。参考自: https://blog.csdn.net/zhaohongfei_358/article/details/122861751

In [47]:
def valid_mask():
  embedding = nn.Embedding(10, 8)
  # 定义Transformer，注意一定要改成eval模型，否则每次输出结果不一样
  transformer = nn.Transformer(d_model=8, batch_first=True).eval()
  src = torch.LongTensor([[0, 1, 2, 3, 4]]) # Encoder的输入
  tgt = torch.LongTensor([[4, 3, 2, 5, 6, 7, 1, 0]]) # Decoder的输入

  def mask_step(step):
    return transformer(embedding(src), embedding(tgt[:, :step]),
            # 这个就是用来生成阶梯式的mask的
            tgt_mask=nn.Transformer.generate_square_subsequent_mask(step))

  print(mask_step(3), mask_step(8), sep='\n')
  with torch.no_grad():
    print(mask_step(8)[:,:3,:] - mask_step(3)) # 两值很接近

valid_mask()

tensor([[[ 0.8515, -1.2293,  0.6525,  1.0049, -0.8316, -1.6763,  0.4246,
           0.8037],
         [ 0.9800, -1.6200,  1.0930,  0.3331, -0.6457, -1.3456,  0.3236,
           0.8815],
         [ 0.8831, -1.5752,  0.3806,  0.3433, -0.6986, -1.3765,  0.9685,
           1.0749]]], grad_fn=<NativeLayerNormBackward0>)
tensor([[[ 0.8515, -1.2293,  0.6525,  1.0049, -0.8316, -1.6763,  0.4246,
           0.8037],
         [ 0.9800, -1.6200,  1.0930,  0.3331, -0.6457, -1.3456,  0.3236,
           0.8815],
         [ 0.8831, -1.5752,  0.3806,  0.3433, -0.6986, -1.3765,  0.9685,
           1.0749],
         [ 1.1004, -0.1711,  0.3520,  1.4578, -0.8099, -1.8714, -0.4478,
           0.3899],
         [ 0.9721, -0.7980,  0.8613,  0.9355, -0.8975, -1.7475, -0.2607,
           0.9348],
         [ 0.4392, -0.7488,  1.1428,  1.0083, -1.0944, -1.4566, -0.4676,
           1.1771],
         [ 0.4680, -1.2360,  1.0205,  1.2123, -1.1367, -1.3567,  0.3134,
           0.7151],
         [ 0.6018, -0.6733, -0.5

8个token经过掩码的前3个的结果 与 只输入3个token经过掩码的结果 很接近

### transformer encoder